# 🧪 Laboratorio 2: Implementación de un Pipeline de Streaming con Kafka

## 🎯 Objetivo

Implementar un pipeline de ingesta en tiempo real en Azure Databricks usando Apache Kafka y Structured Streaming para procesar flujos de datos en tiempo real.

## ℹ️ Introducción

La ingesta de datos en tiempo real es esencial para casos de uso como monitoreo de sensores, análisis de logs o recomendaciones en vivo.
Este laboratorio muestra cómo integrar Apache Kafka o Event Hubs con Databricks para habilitar análisis en tiempo real usando Structured Streaming.

## 🧠 Conceptos Clave

**Apache Kafka**
Sistema distribuido de mensajería altamente escalable usado para procesar flujos de eventos en tiempo real.
[📘 Ver documentación](https://kafka.apache.org/)

**Azure Event Hubs**
Servicio de Azure equivalente a Kafka, compatible con el protocolo Kafka y utilizado para ingesta de datos en tiempo real.
[📘 Ver documentación](https://learn.microsoft.com/en-us/azure/event-hubs/)

**Structured Streaming**
API de Spark para procesamiento incremental y continuo de flujos de datos. Permite tratar flujos como si fueran DataFrames.
[📘 Ver documentación](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)

## 🛠️ Pasos del laboratorio

### 1. Configurar acceso a Kafka o Event Hubs

In [ ]:
kafka_bootstrap_servers = "<broker>:9092"
topic = "eventos"


### 2. Leer datos desde Kafka usando Structured Streaming

In [ ]:
df_kafka = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
    .option("subscribe", topic) \
    .load()

df_valores = df_kafka.selectExpr("CAST(value AS STRING)")

### 3. Procesar datos en tiempo real (ejemplo de parsing JSON)

In [ ]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StringType, IntegerType

schema = StructType() \
    .add("id", IntegerType()) \
    .add("evento", StringType())

df_parsed = df_valores.withColumn("json", from_json(col("value"), schema)) \
    .select("json.*")

### 4. Escribir resultados en consola o Delta Lake (modo append)

In [ ]:
df_parsed.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", "/tmp/checkpoint/kafka") \
    .start("/mnt/datalake/streaming_output")

## ✅ Validación
- Confirma que los datos llegan desde Kafka y son procesados correctamente.
- Verifica que los resultados se almacenen en Delta Lake.
- Monitorea la latencia del stream desde el UI de Spark Structured Streaming.